In [2]:
%pip install great-expectations

   ---------------------------------------- 0.0/5.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.4 MB 640.0 kB/s eta 0:00:09
   ---------------------------------------- 0.1/5.4 MB 525.1 kB/s eta 0:00:11
    --------------------------------------- 0.1/5.4 MB 751.6 kB/s eta 0:00:08
    --------------------------------------- 0.1/5.4 MB 787.7 kB/s eta 0:00:07
   - -------------------------------------- 0.2/5.4 MB 833.5 kB/s eta 0:00:07
   - -------------------------------------- 0.2/5.4 MB 962.7 kB/s eta 0:00:06
   - -------------------------------------- 0.2/5.4 MB 962.7 kB/s eta 0:00:06
   - -------------------------------------- 0.2/5.4 MB 962.7 kB/s eta 0:00:06
   - -------------------------------------- 0.2/5.4 MB 962.7 kB/s eta 0:00:06
   - -------------------------------------- 0.2/5.4 MB 962.7 kB/s eta 0:00:06
   -- ------------------------------------- 0.4/5.4 MB 791.2 kB/s eta 0:00:07
   -- ------------------------------------- 0.4/5.4 MB 791.2 kB/s eta 0:0

In [3]:
import great_expectations as gx
from great_expectations.checkpoint import Checkpoint

In [10]:
#create a Data Context with the default settings
context = gx.get_context()

In [5]:
import sys
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

In [6]:
load_dotenv()
user = os.environ['PG_USER']
password = os.environ['PG_PASSWORD']
host = os.environ['PG_HOST']
port = os.environ['PG_PORT']
database = os.environ['PG_DATABASE']

In [7]:
PG_CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"

In [8]:
#create a Data Source to represent the data available in your PostgreSQL database

pg_datasource = context.sources.add_postgres(
    name="pg_datasource", connection_string=PG_CONNECTION_STRING)

In [11]:
#create a Data Asset to represent a discrete set of data

pg_datasource.add_table_asset(
    name="postgres_telcom_data", table_name="banks_data")

TableAsset(name='tg_post_stats', type='table', id=None, order_by=[], batch_metadata={}, splitter=None, table_name='banks_data', schema_name=None)

In [12]:
#build a Batch Request using the Data Asset you configured above

batch_request = pg_datasource.get_asset("tg_post_stats").build_batch_request()

In [14]:
#build a Batch Request using the Data Asset you configured above

batch_request = pg_datasource.get_asset("postgres_telcom_data").build_batch_request()

In [15]:
expectation_suite_name = "telegram analysis"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

print(validator.head())

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   

In [17]:
validator.expect_column_values_to_not_be_null(column="Last Location Name")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": false,
  "result": {
    "element_count": 150001,
    "unexpected_count": 1153,
    "unexpected_percent": 0.7686615422563849,
    "partial_unexpected_list": [
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null,
      null
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [18]:
validator.expect_column_values_to_be_between(
    column="Start ms", min_value=0, max_value=1000,)

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 150001,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 1,
    "missing_percent": 0.0006666622222518516,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}